In [ ]:
import math

def quadkey_to_tile(quadkey: str):
    x = y = 0
    level = len(quadkey)
    for i in range(level):
        bit = level - i
        mask = 1 << (bit - 1)
        q = int(quadkey[i])
        if q & 1: x |= mask
        if q & 2: y |= mask
    return x, y, level

def tile_to_lonlat_center(x, y, z):
    n = 2 ** z
    lon = x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat = lat_rad * 180.0 / math.pi
    return lon, lat

def quadkey_to_lonlat_center(qk: str):
    x, y, z = quadkey_to_tile(qk)
    return tile_to_lonlat_center(x + 0.5, y + 0.5, z)

# 示例
print(quadkey_to_lonlat_center("1202102330"))